In [1]:
from local_tree_decomp import *

# from local_helpers import *
from help_functions import *

# from local_hom_count import *
# from local_hom_count_int import *
from local_hom_count_best import *
# from local_hom_count_int_dict import *

# from hom_count_bounded_degree import *


In [2]:
graph = graphs.CompleteBipartiteGraph(1, 2)
target_graph = graphs.CompleteGraph(4)

graph_clr = [1, 0, 1]
target_clr = [1, 1, 1, 0]

count_homomorphisms_best(graph, target_graph, graph_clr, target_clr, colourful=True)

6

In [ ]:
%timeit count_homomorphisms(graph, random_graph)

In [ ]:
%load_ext line_profiler

from hom_count_bounded_degree import is_traversal_homomorphism, count_homomorphisms_helper

%lprun -f count_homomorphisms_helper count_homomorphisms_bounded_degree(graph, target_graph)

In [ ]:
def speed_test(graph, target_graph, naive=True):
    basic_result = count_homomorphisms(graph, target_graph)
    int_rep_result = count_homomorphisms_int(graph, target_graph)
    better_int_rep_result = count_homomorphisms_int_pre(graph, target_graph)
    dict_int_rep_result = count_homomorphisms_int_dict(graph, target_graph)

    results = [basic_result, int_rep_result, better_int_rep_result, dict_int_rep_result]

    if naive:
        brute_force_result = len(enumerate_homomorphisms(graph, target_graph))
        results.append(brute_force_result)

    if all(result == better_int_rep_result for result in results):
        print('result:', better_int_rep_result)

        if naive:
            print('\nBrute force:')
            %timeit len(enumerate_homomorphisms(graph, target_graph))
        
        print('\nBasic:')
        %timeit count_homomorphisms(graph, target_graph)
        
        print('\nInt representation:')
        %timeit count_homomorphisms_int(graph, target_graph)
    
        print('\nBetter int representation:')
        %timeit count_homomorphisms_int_pre(graph, target_graph)

        print('\nDict int representation:')
        %timeit count_homomorphisms_int_dict(graph, target_graph)


In [ ]:
g = graphs.CycleGraph(4)

tree_decomp = g.treewidth(certificate=True)
nice_tree_decomp = make_nice_tree_decomposition(g, tree_decomp)
root = sorted(nice_tree_decomp)[0]
dir_labelled_TD = label_nice_tree_decomposition(nice_tree_decomp, root, directed=True)

In [ ]:
g

In [ ]:
for i in dir_labelled_TD.vertices():
    print(i, dir_labelled_TD.get_vertex(i))

In [ ]:
dir_labelled_TD.edges(labels=False)

In [ ]:
dir_labelled_TD

In [ ]:
dir_labelled_TD.vertices()

In [ ]:
node_changes(dir_labelled_TD)

In [ ]:
from local_hom_count_colour import *

graph = graphs.CompleteBipartiteGraph(1, 2)
target_graph = graphs.CompleteGraph(3)

graph_clr = [0, 1, 2]
target_clr = [0, 1, 2]

count_homomorphisms_best_colour(graph, target_graph, graph_clr, target_clr)

In [ ]:
graph

In [ ]:
target_graph

In [ ]:
tree_decomp = graph.treewidth(certificate=True)
nice_tree_decomp = make_nice_tree_decomposition(graph, tree_decomp)
root = sorted(nice_tree_decomp)[0]
dir_labelled_TD = label_nice_tree_decomposition(nice_tree_decomp, root, directed=True)

In [ ]:
dir_labelled_TD

In [3]:
def best_speed_test(graph, target_graph, graph_clr, target_clr):
    # best_result = count_homomorphisms_best(graph, target_graph)
    # best_colour_result = count_homomorphisms_best_colour(graph, target_graph, graph_clr, target_clr)

    print('Best result:')
    %timeit count_homomorphisms_best(graph, target_graph)

    print('\nBest colour result:')
    %timeit count_homomorphisms_best(graph, target_graph, graph_clr, target_clr, colourful=True)


In [4]:
import random
random.seed(int(42))

graph = graphs.CompleteBipartiteGraph(1, 2)
# target_graph = graphs.CompleteGraph(3)

graph_clr = [0, 1, 1]
# target_clr = [0, 1, 2]

for i in range(1, 21):
    N = 10 * i
    # complete_graph = graphs.CompleteGraph(N)
    random_graph = graphs.RandomGNP(N, 0.5, seed=42)
    random_clr = [random.randint(0, 1) for _ in range(N)]
    print('N: ', N)
    best_speed_test(graph, random_graph, graph_clr, random_clr)
    print()

N:  10
Best result:
3.17 ms ± 152 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)

Best colour result:
5.61 ms ± 886 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)

N:  20
Best result:
4.58 ms ± 496 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)

Best colour result:
10.8 ms ± 735 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)

N:  30
Best result:
6.06 ms ± 309 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)

Best colour result:
30.8 ms ± 13.5 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)

N:  40
Best result:
12.4 ms ± 4.02 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)

Best colour result:
54 ms ± 10.2 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)

N:  50
Best result:
15.9 ms ± 6.15 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)

Best colour result:
75.6 ms ± 5.75 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)

N:  60
Best result:
16 ms ± 620 µs per loop (mean ± std. dev. of 7 runs, 100 loo

KeyboardInterrupt: 

In [ ]:
graph = graphs.CompleteBipartiteGraph(1, 3)
graph_clr = [1, 1, 1, 1]

target_graph = graphs.PetersenGraph()

max_num = 0

for _ in range(2):
    random_clr = [1 for _ in range(10)]
    num = count_homomorphisms_best_colour(graph, target_graph, graph_clr, random_clr)
    if num > max_num:
        max_num = num

print(max_num)

In [ ]:
count_homomorphisms_best(graph, target_graph)